# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

In [1]:
import pandas as pd

dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

In [2]:
#Create new features by performing following transformation on inputs:

from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

Using the entire house dataset, learn regression weights using an L1 penalty of 5e2. Make sure to add "normalize=True" when creating the Lasso object. Refer to the following code snippet for the list of features.

**Note**. From here on, the list 'all_features' refers to the list defined in this snippet.

In [7]:
from sklearn import linear_model  # using scikit-learn

all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

**Which features have been chosen by LASSO, i.e. which features were assigned nonzero weights?**

In [8]:
for i in range(len(all_features)):
    print(all_features[i], " : ", model_all.coef_[i])

bedrooms  :  0.0
bedrooms_square  :  0.0
bathrooms  :  0.0
sqft_living  :  134.439313955
sqft_living_sqrt  :  0.0
sqft_lot  :  0.0
sqft_lot_sqrt  :  0.0
floors  :  0.0
floors_square  :  0.0
waterfront  :  0.0
view  :  24750.0045856
condition  :  0.0
grade  :  61749.1030907
sqft_above  :  0.0
sqft_basement  :  0.0
yr_built  :  -0.0
yr_renovated  :  0.0


To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets. Download the provided csv files containing training, validation and test sets.

In [9]:
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)

In [10]:
#Make sure to create the 4 features as we did in #1:

testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

### Now for each l1_penalty in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type np.logspace(1, 7, num=13).)
* Learn a model on TRAINING data using the specified l1_penalty. Make sure to specify normalize=True in the constructor
* Compute the RSS on VALIDATION for the current model (print or save the RSS)

Report which L1 penalty produced the lower RSS on VALIDATION.

In [11]:
import numpy as np

for l1_penalty in np.logspace(1,7, num=13):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price']) # fit with training data
    
    y_hat= model.predict(validation[all_features])
    y_v = validation['price']
    #compute the RSS on Validation for each model
    print("RSS for model", l1_penalty,": %.2f" 
          % np.sum((y_hat-y_v)**2))
    

RSS for model 10.0 : 398213327300134.38
RSS for model 31.6227766017 : 399041900253348.50
RSS for model 100.0 : 429791604072558.44
RSS for model 316.227766017 : 463739831045119.44
RSS for model 1000.0 : 645898733633803.25
RSS for model 3162.27766017 : 1222506859427156.75
RSS for model 10000.0 : 1222506859427156.75
RSS for model 31622.7766017 : 1222506859427156.75
RSS for model 100000.0 : 1222506859427156.75
RSS for model 316227.766017 : 1222506859427156.75
RSS for model 1000000.0 : 1222506859427156.75
RSS for model 3162277.66017 : 1222506859427156.75
RSS for model 10000000.0 : 1222506859427156.75


### Now that you have selected an L1 penalty, compute the RSS on TEST data for the model with the best L1 penalty.

In [13]:
l1_penalty =10.0
model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
model.fit(training[all_features], training['price']) # fit with training data

y_hat= model.predict(testing[all_features])
y_v = testing['price']
#compute the RSS on Validation for each model
print("RSS for model", l1_penalty,": %.2f" 
      % np.sum((y_hat-y_v)**2))

RSS for model 10.0 : 98467402552698.86


**Using the best L1 penalty, how many nonzero weights do you have? Count the number of nonzero coefficients first, and add 1 if the intercept is also nonzero. **

In [14]:
np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)

15

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [15]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [45]:
l1_penalty_values = list(np.logspace(1, 4, num=20))

##### For l1_penalty in np.logspace(1, 4, num=20):
* Fit a regression model with a given l1_penalty on TRAIN data. Add "alpha=l1_penalty" and "normalize=True" to the parameter list. 
* Extract the weights of the model and count the number of nonzeros. Take account of the intercept as we did in #8, adding 1 whenever the intercept is nonzero. Save the number of nonzeros to a list.

In [71]:
nonzeros = {}
for l1_penalty in l1_penalty_values:
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price'])
    nonzeros.update({l1_penalty :(np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_))})

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [72]:
nonzeros

{10.0: 15,
 14.384498882876629: 15,
 20.691380811147901: 15,
 29.763514416313178: 15,
 42.813323987193932: 13,
 61.584821106602639: 12,
 88.586679041008225: 11,
 127.42749857031335: 10,
 183.29807108324357: 7,
 263.66508987303581: 6,
 379.26901907322497: 6,
 545.55947811685144: 6,
 784.75997035146065: 5,
 1128.8378916846884: 3,
 1623.776739188721: 3,
 2335.7214690901214: 2,
 3359.8182862837812: 1,
 4832.9302385717519: 1,
 6951.9279617756056: 1,
 10000.0: 1}

In [73]:
l_min=[]
l_max=[]
for a,b in nonzeros.items():
    if b>max_nonzeros:
        l_max.append(a)
    if b<max_nonzeros:
        l_min.append(a)

In [89]:
l1_penalty_max = min(l_min)
l1_penalty_min = max(l_max) 
print('l1_penalty_min: ', l1_penalty_min, '\n', 'l1_penalty_max :',l1_penalty_max) 

l1_penalty_min:  127.42749857 
 l1_penalty_max : 263.665089873


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [79]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given l1_penalty on TRAIN data. As before, use "alpha=l1_penalty" and "normalize=True"
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity equal to ‘max_nonzeros’. (Again, take account of the intercept when counting the number of nonzeros.)

In [80]:
pr = {}
for l1_penalty in l1_penalty_values:
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price'])
    #measure the RSS on the Validation set
    y_hat= model.predict(validation[all_features])
    y_v = validation['price']
    #compute the RSS on Validation for each model
    RSS= np.sum((y_hat-y_v)**2)
    sparsity = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    if sparsity == max_nonzeros:
        pr.update({l1_penalty: RSS})

**What value of l1_penalty in our narrow range has the lowest RSS on the VALIDATION set and has sparsity equal to ‘max_nonzeros’?**

In [82]:
b_min=1e50
for a,b in pr.items():
    if b<b_min:
        b_min=b
        l1_low=a
        
print(b_min)
print(l1_low)

440037365263316.94
156.109096739


** What features in this model have non-zero coefficients?**

In [86]:
model = linear_model.Lasso(alpha=l1_low, normalize=True)
model.fit(training[all_features], training['price'])
for i in range(len(all_features)):
    print(all_features[i], " : ", model.coef_[i])

bedrooms  :  -0.0
bedrooms_square  :  -0.0
bathrooms  :  10610.8902844
sqft_living  :  163.380251648
sqft_living_sqrt  :  0.0
sqft_lot  :  -0.0
sqft_lot_sqrt  :  -0.0
floors  :  0.0
floors_square  :  0.0
waterfront  :  506451.687115
view  :  41960.0435549
condition  :  0.0
grade  :  116253.5537
sqft_above  :  0.0
sqft_basement  :  0.0
yr_built  :  -2612.23488036
yr_renovated  :  0.0
